In [11]:
from google.colab import drive
import os
import re
import tifffile
from datetime import datetime
from shutil import copyfile

# Mount Google Drive
drive.mount('/content/drive')

# Define folders
input_folder = '/content/drive/My Drive/EOD_2_SN/EOD-2/'
output_folder = '/content/drive/My Drive/EOD_2_SN/SN/'
os.makedirs(output_folder, exist_ok=True)

# Helper to sanitize strings for filenames
def clean(text):
    return re.sub(r'[^\w\-]', '_', str(text)).strip()

index = 1

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith('.tif'):
        continue

    original_path = os.path.join(input_folder, filename)

    try:
        with tifffile.TiffFile(original_path) as tif:
            cz_sem = tif.pages[0].tags.get('CZ_SEM')
            if cz_sem is None:
                print(f"⚠️ No CZ_SEM metadata in {filename}")
                continue
            metadata = cz_sem.value if isinstance(cz_sem.value, dict) else tif.pages[0].tags['CZ_SEM'].value

        # Extract metadata fields safely
        dp_detector_type = clean(metadata.get('dp_detector_type', ['Detector', 'UNKNOWN'])[1])
        ap_date_raw = metadata.get('ap_date', ['Date', 'UNKNOWN'])[1]
        try:
            ap_date = datetime.strptime(ap_date_raw, "%d %b %Y").strftime("%Y%m%d")
        except:
            ap_date = "UNKNOWNDATE"
        sv_file_name = clean(metadata.get('sv_file_name', ['File Name', filename])[1].replace('.tif', ''))
        ap_mag_val = metadata.get('ap_mag', ['Mag', ''])[1]
        ap_mag = f"{ap_mag_val}X" if isinstance(ap_mag_val, (int, float)) else clean(str(ap_mag_val))

        # Create new filename
        new_name = f"{dp_detector_type}_{ap_date}_{sv_file_name}_{ap_mag}_{index:03d}.tif"
        new_path = os.path.join(output_folder, new_name)

        # Copy (not move)
        copyfile(original_path, new_path)
        print(f"✅ Copied: {filename} → {new_name}")
        index += 1

    except Exception as e:
        print(f"❌ Error with {filename}: {e}")

Mounted at /content/drive
✅ Copied: 10-11-60.tif → SE2_20250324_10-11-60_30_00_K_X_001.tif
✅ Copied: 10-12-59.tif → SE2_20250324_10-12-59_30_00_K_X_002.tif
✅ Copied: 10-15-53.tif → SE2_20250324_10-15-53_30_00_K_X_003.tif
✅ Copied: 10-16-56.tif → SE2_20250324_10-16-56_30_00_K_X_004.tif
✅ Copied: 10-3-68.tif → SE2_20250324_10-3-68_30_00_K_X_005.tif
✅ Copied: 10-4-67.tif → SE2_20250324_10-4-67_30_00_K_X_006.tif
✅ Copied: 10-7-64.tif → SE2_20250324_10-7-64_30_00_K_X_007.tif
✅ Copied: 10-8-63.tif → SE2_20250324_10-8-63_30_00_K_X_008.tif
✅ Copied: 20--20-36.tif → SE2_20250321_20--2036_30_00_K_X_009.tif
✅ Copied: 20--24-37.tif → SE2_20250321_20--24-37_30_00_K_X_010.tif
✅ Copied: 20-19-41.tif → SE2_20250324_20-19-41_30_00_K_X_011.tif
✅ Copied: 20-23-44.tif → SE2_20250324_20-23-44_30_00_K_X_012.tif
✅ Copied: 20-27-47.tif → SE2_20250324_20-27-47_30_00_K_X_013.tif
✅ Copied: 20-28-38.tif → SE2_20250321_20-28-38_30_00_K_X_014.tif
✅ Copied: 20-31-52.tif → SE2_20250324_20-31-52_28_09_K_X_015.tif
✅ Co